In [3]:
import sys

sys.path.append("./src")

from pipeline import GCLPipeline

import os
import os.path as osp
import logging
import json
import pandas as pd

import torch
import torch_geometric.transforms as T

from torch.optim import Adam

from tqdm import tqdm

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

import logging

# from importlib import reload
# reload(logging)
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG, datefmt="%I:%M:%S"
)


DATA_PATH = "./data"
BENCHMARK_PATH = "./benchmark"

OUTPUT_FOLDER = "./output"

In [33]:
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark-infograph.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark-dgi.json")))
STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark-grace.json")))

# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-infonce.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-jsd.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-triplet.json"))) # NOT WORKING

In [34]:
CONFIG = STRATEGIES["config"]
TRY_GPU = CONFIG["try_gpu"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() and TRY_GPU else "cpu")

In [35]:
outputs = []

for strategy in STRATEGIES["strategies"]:

    logger.info({strategy["_descr"]})

    # Initialization
    pipeline = GCLPipeline.from_strategy(strategy, DEVICE)

    # Data
    dataset, num_features = GCLPipeline.init_dataset(
        strategy["dataset"], DATA_PATH, T.NormalizeFeatures(), strategy["batch_size"]
    )

    # Encoder
    encoder_params = strategy["encoder_params"]
    encoder_params["input_dim"] = num_features
    encoder_model = pipeline.init_encoder(encoder_params, DEVICE)

    # Training
    train_params = strategy["train_params"]
    lr = train_params["learning_rate"]
    epochs = train_params["epochs"]

    optimizer = Adam(encoder_model.parameters(), lr=lr)
    losses = []

    with tqdm(total=epochs, desc="(T)") as pbar:
        for epoch in tqdm(range(1, epochs + 1)):
            loss = pipeline.train_epoch(encoder_model, dataset, optimizer, DEVICE)
            pbar.set_postfix({"loss": loss})

            pbar.update()

    test_result = pipeline.test(encoder_model, dataset, DEVICE)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')

    outputs.append(
        {
            "Data": strategy["dataset"],
            "Method": strategy["method"],
            "Archi": strategy["architecture"],
            "Mode": strategy["mode"],
            "Obj": strategy["objective"],
            "Neg": strategy["negative"],
            "Aug1": (
                strategy["augmentation1"]["name"]
                if isinstance(strategy["augmentation1"], dict)
                else ', '.join([a["name"] for a in strategy["augmentation1"]])
            ),
            "Aug1Strat": strategy["augmentation1_strat"],
            "Aug2": (
                strategy["augmentation2"]["name"]
                if isinstance(strategy["augmentation2"], dict)
                else ', '.join([a["name"] for a in strategy["augmentation2"]])
            ),
            "Aug2Strat": strategy["augmentation2_strat"],
            "MicroF1": test_result["micro_f1"].round(2),
            "MacroF1": test_result["macro_f1"].round(2),
        }
    )

11:52:02 INFO:{'GRACE - L2L - INFONCE - Default'}
11:52:02 INFO:CALL GCLPipeline.from_strategy
11:52:02 INFO:	 Method: GRACE
11:52:02 INFO:	 Augmentation strategy 1: Compose
11:52:02 INFO:	 Augmentation strategy 2: Compose
11:52:02 INFO:CALL GCLPipeline.init_augmentations
11:52:02 INFO:	 Strategy: Compose
11:52:02 INFO:CALL GCLPipeline.init_augmentation
11:52:02 INFO:	 Name: EdgeRemoving
11:52:02 INFO:CALL GCLPipeline.init_augmentation
11:52:02 INFO:	 Name: FeatureMasking
11:52:02 INFO:CALL GCLPipeline.init_augmentations
11:52:02 INFO:	 Strategy: Compose
11:52:02 INFO:CALL GCLPipeline.init_augmentation
11:52:02 INFO:	 Name: EdgeRemoving
11:52:02 INFO:CALL GCLPipeline.init_augmentation
11:52:02 INFO:	 Name: FeatureMasking
11:52:02 INFO:Augmentations: [<GCL.augmentors.augmentor.Compose object at 0x7f8a023f62f0>, <GCL.augmentors.augmentor.Compose object at 0x7f8a023f6d10>]
11:52:02 INFO:CALL GCLPipeline.init_dataset
11:52:02 INFO:	 Number of features: 1433
11:52:02 INFO:CALL GCLPipeline.i

(T):  70%|████████████████▊       | 700/1000 [00:19<00:08, 36.64it/s, loss=4.28]


(LR): 100%|██████████| 5000/5000 [00:08<00:00, best test F1Mi=0.761, F1Ma=0.749]
11:52:37 INFO:{'GRACE - L2L - INFONCE - No Feature Masking'}
11:52:37 INFO:CALL GCLPipeline.from_strategy
11:52:37 INFO:	 Method: GRACE
11:52:37 INFO:	 Augmentation strategy 1: Compose
11:52:37 INFO:	 Augmentation strategy 2: Compose
11:52:37 INFO:CALL GCLPipeline.init_augmentations
11:52:37 INFO:	 Strategy: Compose
11:52:37 INFO:CALL GCLPipeline.init_augmentation
11:52:37 INFO:	 Name: EdgeRemoving
11:52:37 INFO:CALL GCLPipeline.init_augmentations
11:52:37 INFO:	 Strategy: Compose
11:52:37 INFO:CALL GCLPipeline.init_augmentation
11:52:37 INFO:	 Name: EdgeRemoving
11:52:37 INFO:CALL GCLPipeline.init_augmentation
11:52:37 INFO:	 Name: FeatureMasking
11:52:37 INFO:Augmentations: [<GCL.augmentors.augmentor.Compose object at 0x7f8a2db1cf40>, <GCL.augmentors.augmentor.Compose object at 0x7f8a2db1c670>]
11:52:37 INFO:CALL GCLPipeline.init_dataset
11:52:38 INFO:	 Number of features: 1433
11:52:38 INFO:CALL GCLPipe

(E): Best test F1Mi=0.7610, F1Ma=0.7492


  0%|                                                  | 0/1000 [00:00<?, ?it/s]/home/felix/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(T):  37%|████████▉               | 372/1000 [00:10<00:16, 37.48it/s, loss=4.37]


...

11:58:42 INFO:	 Activation: ReLU
11:58:42 INFO:CALL GRACEEncoder
11:58:42 INFO:Encoder: GRACEGConv(
  (activation): ReLU()
  (layers): ModuleList(
    (0): GCNConv(1433, 32)
    (1): GCNConv(32, 32)
  )
)
11:58:42 INFO:Augmentor: (<GCL.augmentors.augmentor.Compose object at 0x7f8a02425780>, <GCL.augmentors.augmentor.Compose object at 0x7f8a024253f0>)


(E): Best test F1Mi=0.4265, F1Ma=0.2924


  0%|                                                  | 0/1000 [00:00<?, ?it/s]/home/felix/.local/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(T):  56%|████████████▊          | 556/1000 [00:10<00:08, 52.40it/s, loss=0.221]


(LR): 100%|██████████| 5000/5000 [00:08<00:00, best test F1Mi=0.397, F1Ma=0.274]

(E): Best test F1Mi=0.3971, F1Ma=0.2736


# Result DataFrame

In [36]:
output_df = pd.DataFrame(outputs)
output_df

,Data,Method,Archi,Mode,Obj,Neg,Aug1,Aug1Strat,Aug2,Aug2Strat,MicroF1,MacroF1
0,Cora,GRACE,DualBranch,L2L,InfoNCE,None,"EdgeRemoving, FeatureMasking",Compose,"EdgeRemoving, FeatureMasking",Compose,0.76,0.75
1,Cora,GRACE,DualBranch,L2L,InfoNCE,None,EdgeRemoving,Compose,"EdgeRemoving, FeatureMasking",Compose,0.77,0.77
2,Cora,GRACE,DualBranch,L2L,InfoNCE,None,"EdgeRemoving, FeatureMasking, NodeDropping",Compose,"EdgeRemoving, FeatureMasking",Compose,0.76,0.75
3,Cora,GRACE,DualBranch,L2L,JSD,None,"EdgeRemoving, FeatureMasking",Compose,"EdgeRemoving, FeatureMasking",Compose,0.63,0.61
4,Cora,GRACE,DualBranch,L2L,JSD,None,EdgeRemoving,Compose,"EdgeRemoving, FeatureMasking",Compose,0.78,0.74
5,Cora,GRACE,DualBranch,L2L,JSD,None,"EdgeRemoving, FeatureMasking, NodeDropping",Compose,"EdgeRemoving, FeatureMasking",Compose,0.66,0.58
6,Cora,GRACE,DualBranch,L2L,BarlowTwins,None,"EdgeRemoving, FeatureMasking",Compose,"EdgeRemoving, FeatureMasking",Compose,0.82,0.78
7,Cora,GRACE,DualBranch,L2L,BarlowTwins,None,EdgeRemoving,Compose,"EdgeRemoving, FeatureMasking",Compose,0.76,0.74
8,Cora,GRACE,DualBranch,L2L,BarlowTwins,None,"EdgeRemoving, FeatureMasking, NodeDropping",Compose,"EdgeRemoving, FeatureMasking",Compose,0.80,0.77
9,Cora,GRACE,DualBranch,L2L,BootstrapLatent,None,"EdgeRemoving, FeatureMasking",Compose,"EdgeRemoving, FeatureMasking",Compose,0.42,0.33


# Saving

In [37]:
# output_file = "output.csv"
# output_file = "augmentation_benchmark_graphcl_infonce.csv"
# output_file = "augmentation_benchmark_graphcl_jsd.csv"
output_file = "augmentation_benchmark_grace.csv"
output_path = osp.join(OUTPUT_FOLDER, output_file)
output_path

'./output/augmentation_benchmark_grace.csv'

In [38]:
# Deleting output file
# DO NOT REMOVE BEFORE MAKING A COPY
# os.remove(OUTPUT_PATH)

In [39]:
# Adding rows
output_df.to_csv(output_path, mode='a', header=not osp.exists(output_path), index=False)